In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import time

from copy import deepcopy
from numpy import diag, exp, log, sqrt, trace
from numpy.linalg import eig, cholesky, inv, norm, solve, svd
from scipy.linalg import expm, solve_lyapunov, solve_sylvester, sqrtm
from scipy.sparse import diags

In [ ]:
# does not work for larger dimensions -> Lyapunov solver does not converge

D = 50
r = 2
d = 2 * D

#L1 = np.eye(d) + 0.01 * np.random.randn(d, d)
#L2 = np.eye(d) + 0.01 * np.random.randn(d, d)
#F_ = np.eye(d) + 0.01 * np.random.randn(d, d)

#K = L1.dot(L1.T)
#M = L2.dot(L2.T)
#F = F_.dot(F_.T)

#K_ = 0.01 * np.random.randn(d, d)
#M_ = 0.01 * np.random.randn(d, d)
#F_ = 0.01 * np.random.randn(d, d)
#K = 0.5 * (K_ + K_.T)  + np.diag([.1] * r + [1] * (d - r)) #np.eye(d)
#M = 0.5 * (M_ + M_.T)  + np.diag([.1] * r + [1] * (d - r)) #np.eye(d)
#F = F_.dot(F_.T)  + np.diag([.01] * r + [.1] * (d - r)) #np.eye(d)

K = np.eye(D) + np.abs(0.1 * np.random.randn(D, D))
M = np.eye(D) + np.diag(np.abs(0.5 * np.random.randn(D)))
F = 2 * (np.eye(D) + np.diag(0.1 * np.random.randn(D)))

L1 = cholesky(K)
L2 = cholesky(M)

if ~np.all(eig(K)[0] > 0) or ~np.all(eig(M)[0] > 0) or ~np.all(eig(F)[0] > 0):
    print('not all eigenvalues positive')

A = np.zeros([2 * D, 2 * D])
A[0:D, D:2 * D] = L1.T.dot(inv(L2).T)
A[D:2 * D, 0:D] = -inv(L2).dot(L1)
A[D:2 * D, D: 2 * D] = -inv(L2).dot(F).dot(inv(L2).T)

Gamma_0 = np.eye(d) # + 0.1 * np.random.randn(d, d)

N = np.zeros([2 * D, 2 * D])
N[D:2 * D, D: 2 * D] = Gamma_0

B = np.zeros([2 * D, 2 * D])
B[D:2 * D, D: 2 * D] = Gamma_0

if ~np.all(np.real(eig(A)[0]) < 0):
    print('not all eigenvalues are negative')

def u_t(x, t):
    #return np.zeros(x.shape)
    return np.sin(20 * t) * np.ones(x.shape)

In [ ]:
T_end = 10
tau = 0.1
delta_t = 0.001
N_end = int(T / delta_t)
tau_n = int(tau / delta_t)
gamma = 10

Y = np.zeros([N_end + 1, 2 * D])
Y[0, :] = np.ones(2 * D)

for n in range(N_end):
    u = u_t(Y[n, :], n * delta_t)
    if n >= tau_n:
        Y_delay = Y[n - tau_n, :]
    else:
        Y_delay = np.zeros(2 * d)
    Y[n + 1, :] = Y[n, :] + (A.dot(Y[n, :]) + tau / 2 * gamma * N.dot(Y_delay) + B.dot(u)) * delta_t

In [ ]:
plt.plot(np.linspace(0, T_end, N_end + 1), inv(L1).T.dot(Y[:, :D].T).T[:, 0]);
plt.plot(np.linspace(0, T_end, N_end + 1), inv(L1).T.dot(Y[:, :D].T).T[:, 1]);
plt.plot(np.linspace(0, T_end, N_end + 1), inv(L1).T.dot(Y[:, :D].T).T[:, 2]);

In [ ]:
C = np.eye(2 * D)
B_in = np.ones([2 * D, 2 * D])

I = 300
Q_i = np.zeros([I + 1, 2 * D, 2 * D])
Q_i[0, :, :] = solve_lyapunov(A.T, -C.T.dot(C))
for i in range(I):
    Q_i[i + 1, :, :] = solve_lyapunov(A.T, -N.T.dot(Q_i[i, :, :]).dot(N))
Q = np.sum(Q_i, 0)

I = 300
P_i = np.zeros([I + 1, 2 * D, 2 * D])
P_i[0, :, :] = solve_lyapunov(A, -B_in.dot(B_in.T))
for i in range(I):
    P_i[i + 1, :, :] = solve_lyapunov(A, -N.dot(P_i[i, :, :]).dot(N.T))
P = np.sum(P_i, 0)

if np.sum(P_i[-1, :, :]) > 1e-6 or np.sum(Q_i[-1, :, :]) > 1e-6:
    print('Lyapunov solver did not converge')

#K = sqrtm(P)
#L = sqrtm(Q)

V_P, sigmas_P, U_P_T = svd(P)
K = V_P.dot(sqrt(diag(sigmas_P))).dot(U_P_T)

V_Q, sigmas_Q, U_Q_T = svd(Q)
L = V_Q.dot(sqrt(diag(sigmas_Q))).dot(U_Q_T)

V, sigmas, U_T = svd(K.T.dot(L))
Sigma = np.diag(sigmas)
Sigma_inv_05 = np.diag(1 / sqrt(sigmas))

T = Sigma_inv_05.dot(U_T).dot(L.T)
T_inv = K.dot(V).dot(Sigma_inv_05)
A_ = T.dot(A).dot(T_inv)
B_ = T.dot(B)
N_ = T.dot(N).dot(T_inv)
C_ = C.dot(T_inv)

plt.scatter(range(1, 2 * D + 1), sigmas) # (Hankel singular values)
plt.yscale('log');
plt.ylim(0.8 * min(sigmas), 1.2 * max(sigmas));

In [ ]:
np.max(eig(N)[0])

In [ ]:
np.max(eig(N)[0]) / np.sqrt(-2*np.max(np.real(eig(A)[0])))

In [ ]:
plt.plot([np.sum(P_i[i, :, :]) for i in range(I + 1)]);
plt.yscale('log')

In [ ]:
r = 2

A_11 = A_[:r, :r]
A_12 = A_[:r, r:]
A_21 = A_[r:, :r]
A_22 = A_[r:, r:]
B_11 = B_[:r, :r]
B_12 = B_[:r, r:]
B_21 = B_[r:, :r]
B_22 = B_[r:, r:]
N_11 = N_[:r, :r]
N_12 = N_[:r, r:]
N_21 = N_[r:, :r]
N_22 = N_[r:, r:]
C_1 = C_[:, :r]
C_2 = C_[:, r:]

# Balanced truncation
A_r = A_11
B_r = B_11
N_r = N_11
C_r = C_1

T_end = 10
tau = 0.1
delta_t = 0.001
N_end = int(T_end / delta_t)
tau_n = int(tau / delta_t)
gamma = 10

Y = np.zeros([N_end + 1, d])
Y_r = np.zeros([N_end + 1, r])
Y[0, :] = np.ones(d)
Y_r[0, :] = T.dot(Y[0, :])[:r]

for n in range(N_end):
    u = u_t(Y[n, :], n * delta_t)
    u_r = u_t(Y_r[n, :], n * delta_t)
    if n >= tau_n:
        Y_delay = Y[n - tau_n, :]
        Y_r_delay = Y_r[n - tau_n, :]
    else:
        Y_delay = np.zeros(d)
        Y_r_delay = np.zeros(r)
    Y[n + 1, :] = Y[n, :] + (A.dot(Y[n, :]) + tau / 2 * gamma * N.dot(Y_delay) + B.dot(u)) * delta_t
    Y_r[n + 1, :] = Y_r[n, :] + (A_r.dot(Y_r[n, :]) + tau / 2 * gamma * N_r.dot(Y_r_delay) + B_r.dot(u_r)) * delta_t
    
CY_r = C_r.dot(Y_r.T).T

#Y_2 = deepcopy(Y)
#CY_r_2 = deepcopy(CY_r)

In [ ]:
COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:olive',
          'tab:pink', 'tab:gray', 'tab:brown', 'tab:cyan']

fig, ax = plt.subplots(1, 2, figsize=(12, 3.5))


t_val = np.linspace(0, T_end, N_end + 1)

ax[0].set_title(r'Full and reduced trajectories, $d = %d, r = %d$' % (d, 2))
for i, j in enumerate([0, 1, 3, 4, 5]):
    ax[0].plot(t_val, inv(L1).T.dot(Y[:, :D].T).T[:, j], color=COLORS[i])
    ax[0].plot(t_val, inv(L1).T.dot(CY_r[:, :D].T).T[:, j], '--', color=COLORS[i]);
ax[0].set_xlabel(r'$t$')

ax[1].set_title(r'Full and reduced trajectories, $d = %d, r = %d$' % (d, 10))
for i, j in enumerate([0, 1, 3, 4, 5]):
    ax[1].plot(t_val, inv(L1).T.dot(Y_2[:, :D].T).T[:, j], color=COLORS[i])
    ax[1].plot(t_val, inv(L1).T.dot(CY_r_2[:, :D].T).T[:, j], '--', color=COLORS[i]);
ax[1].set_xlabel(r'$t$');
    
#fig.savefig('img/Langevin_d100_T20.pdf')

In [ ]:
t_val = np.linspace(0, T_end, N_end + 1)

COLORS = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:olive',
          'tab:pink', 'tab:gray', 'tab:brown', 'tab:cyan']

plt.title(r'$d = %d, r = %d, \Delta t = %.1e$' % (2 * d, r, delta_t))
for i, j in enumerate(np.random.choice(range(d), 3, replace=False)):
    plt.plot(t_val, inv(L1).T.dot(Y[:, :d].T).T[:, j], color=COLORS[i])
    plt.plot(t_val, inv(L1).T.dot(CY_r[:, :d].T).T[:, j], '--', color=COLORS[i]);

In [ ]:
L2_errors = []
bounds_thm2 = []

D = 50
    
for r in range(1, d):
    A_11 = A_[:r, :r]
    A_12 = A_[:r, r:]
    A_21 = A_[r:, :r]
    A_22 = A_[r:, r:]
    B_11 = B_[:r, :r]
    B_12 = B_[:r, r:]
    B_21 = B_[r:, :r]
    B_22 = B_[r:, r:]
    N_11 = N_[:r, :r]
    N_12 = N_[:r, r:]
    N_21 = N_[r:, :r]
    N_22 = N_[r:, r:]
    C_1 = C_[:, :r]
    C_2 = C_[:, r:]

    # Balanced truncation
    A_r = A_11
    B_r = B_11
    N_r = N_11
    C_r = C_1
    
    T_end = 2
    tau = 0.1
    delta_t = 0.001
    N_end = int(T_end / delta_t)
    tau_n = int(tau / delta_t)
    gamma = 10
    
    d = 2 * D

    Y = np.zeros([N_end + 1, d])
    Y_r = np.zeros([N_end + 1, r])
    Y[0, :] = np.ones(d)
    Y_r[0, :] = T.dot(Y[0, :])[:r]

    for n in range(N_end):
        u = u_t(Y[n, :], n * delta_t)
        u_r = u_t(Y_r[n, :], n * delta_t)
        if n >= tau_n:
            Y_delay = Y[n - tau_n, :]
            Y_r_delay = Y_r[n - tau_n, :]
        else:
            Y_delay = np.zeros(d)
            Y_r_delay = np.zeros(r)
        Y[n + 1, :] = Y[n, :] + (A.dot(Y[n, :]) + tau / 2 * gamma * N.dot(Y_delay) + B.dot(u)) * delta_t
        Y_r[n + 1, :] = Y_r[n, :] + (A_r.dot(Y_r[n, :]) + tau / 2 * gamma * N_r.dot(Y_r_delay) + B_r.dot(u_r)) * delta_t

    CY_r = C_r.dot(Y_r.T).T
    
    L2_error = np.sqrt(np.sum(np.sum((Y - np.real(CY_r))**2, 0)) * delta_t)
    L2_errors.append(L2_error)
    
    A_hat = np.zeros([d + r, d + r])
    A_hat[0:d, 0:d] = A
    A_hat[d:d + r, d:d + r] = A_r
    B_in_hat = np.concatenate([B_in, T.dot(B_in)[:r, :]])
    C_hat = np.concatenate([C, -C_r], axis=1)
    N_hat = np.zeros([d + r, d + r])
    N_hat[0:d, 0:d] = np.sqrt(T_end) * N
    N_hat[d:d + r, d:d + r] = np.sqrt(T_end) * N_r

    I = 2 * d
    Q_hat_i = np.zeros([I + 1, d + r, d + r])
    Q_hat_i[0, :, :] = solve_lyapunov(A_hat.T, -C_hat.T.dot(C_hat))
    for i in range(I):
        Q_hat_i[i + 1, :, :] = solve_lyapunov(A_hat.T, -N_hat.T.dot(Q_hat_i[i, :, :]).dot(N_hat))
    Q_hat = np.sum(Q_hat_i, 0)

    I = 2 * d
    P_hat_i = np.zeros([I + 1, d + r, d + r])
    P_hat_i[0, :, :] = solve_lyapunov(A_hat, -B_in_hat.dot(B_in_hat.T))
    for i in range(I):
        P_hat_i[i + 1, :, :] = solve_lyapunov(A_hat, -N_hat.dot(P_hat_i[i, :, :]).dot(N_hat.T))
    P_hat = np.sum(P_hat_i, 0)
    
    if np.sum(P_hat_i[-1, :, :]) > 1e-6 or np.sum(Q_hat_i[-1, :, :]) > 1e-6:
        print('Lyapunov solver did not converge')

    #K_hat = sqrtm(P_hat)
    #L_hat = sqrtm(Q_hat)

    V_P_hat, sigmas_P_hat, U_P_hat_T = svd(P_hat)
    K_hat = V_P_hat.dot(sqrt(diag(sigmas_P_hat))).dot(U_P_hat_T)

    V_Q_hat, sigmas_Q_hat, U_Q_hat_T = svd(Q_hat)
    L_hat = V_Q_hat.dot(sqrt(diag(sigmas_Q_hat))).dot(U_Q_hat_T)

    V_hat, sigmas_hat, U_T_hat = svd(K_hat.T.dot(L_hat))
    Sigma_hat = np.diag(sigmas_hat)
    Sigma_inv_05 = np.diag(1 / sqrt(sigmas_hat))
    
    bounds_thm2.append(4 * sum(sigmas_hat) * np.sqrt(np.sum(Y[0, :]**2)))
